In [3]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

## Determining correlation between time and light_conditions

In [4]:
light = accidents[['converted_date', 'converted_time', 'decimal_time', 'light_conditions']]
light = light.dropna(axis=0)
light.head(5)

light['month'] = light['converted_date'].dt.month

#pearsonr(light['decimal_time'], light['light_conditions'])

light.head(5)

NameError: name 'accidents' is not defined

In [5]:
# pearson correlation of time and light_conditions by month

for i in np.sort(light['month'].unique()):
    month_data = light[light['month'] == i]
    p = pearsonr(month_data['decimal_time'], month_data['light_conditions'])
    print(f'Pearson correlation for month {i}: \n{p}\n')

NameError: name 'light' is not defined

## Determining sunrise and sunset

Determining sunrise and sunset times in London for every day in 2019

In [10]:
# url containing sunrise and sunset data
url = 'https://www.sunrise-and-sunset.com/en/sun/united-kingdom/london/2019/'
r = requests.get(url)

# instantiate BeautifulSoup object
soup = BeautifulSoup(r.content, 'html.parser')

# find all month tables
tables = soup.find_all('table', 
                   attrs = {'class': 'table table-striped table-hover well'})
data = []

# iterate through month tables
for table in tables:
    # find the table body
    body = table.find('tbody')
    # find all table data
    tds = body.find_all('td')
    # iterate through table data
    for td in tds:
        # append the table data to the data list
        data.append(td.get_text())

# convert to numpy.ndarray and reshape
data = np.array(data).reshape(-1,4)
# convert back to standard python list
data = data.tolist()

# add headers
data.insert(0, ['date', 'sunrise', 'sunset', 'day_length'])

# write data list to sunrise_sunset.csv
with open('sunrise_sunset.csv', 'w') as f:
    write = csv.writer(f)
    for row in data:
        write.writerow(row)
        
suntimes = pd.read_csv('sunrise_sunset.csv')


suntimes

,date,sunrise,sunset,day_length
0,"1 January 2019, Tuesday",08:06:00,16:01:00,07:54:00
1,"2 January 2019, Wednesday",08:06:00,16:02:00,07:55:00
2,"3 January 2019, Thursday",08:06:00,16:03:00,07:56:00
3,"4 January 2019, Friday",08:06:00,16:04:00,07:58:00
4,"5 January 2019, Saturday",08:06:00,16:05:00,07:59:00
...,...,...,...,...
360,"27 December 2019, Friday",08:06:00,15:56:00,07:49:00
361,"28 December 2019, Saturday",08:06:00,15:57:00,07:50:00
362,"29 December 2019, Sunday",08:06:00,15:58:00,07:51:00
363,"30 December 2019, Monday",08:06:00,15:59:00,07:52:00


In [7]:
# read sunset_sunrise.csv into a dataframe
suntimes = pd.read_csv('sunrise_sunset.csv')

# # convert datatype of columns to datetime
# for col in suntimes.columns:
#     if col == 'date':
#         suntimes[col] = pd.to_datetime(suntimes[col])
#     else:
#         suntimes[col] = pd.to_datetime(suntimes[col],
#                                         errors='raise',
#                                         format='%H:%M').dt.time
        
# create a single 'datetime' column in ISO 8601 format
# using numpy.datetime64 object
sunrise = suntimes.date + ' ' + suntimes.sunrise
suntimes['converted_sunrise'] = pd.DatetimeIndex(sunrise)    
sunset = suntimes.date + ' ' + suntimes.sunset
suntimes['converted_sunset'] = pd.DatetimeIndex(sunset)
#convert_to_decimal(suntimes, 'sunrise_decimal', 'sunrise')
day_length = suntimes.date + ' ' + suntimes['day_length']
suntimes['converted_daylength'] = pd.DatetimeIndex(day_length)
convert_to_decimal(suntimes, 'daylength_decimal', 'converted_daylength')
a = suntimes['daylength_decimal'] / 2
b = a
suntimes.head(5)

suntimes['diff'] = suntimes['converted_sunset'] - suntimes['converted_sunrise']

suntimes

NameError: name 'convert_to_decimal' is not defined

In [ ]:
url = 'https://www.worldfootball.net/attendance/eng-premier-league-2018-2019/1/'
r = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

table = soup.find('table',
                 attrs={'class': 'standard_tabelle'})

data = []

tds = table.find_all('td')
for td in tds:
    data.append(td.get_text().strip())
    
data = [i for i in data if i]
data = np.array(data).reshape(-1, 5)

data = data.tolist()

# add headers
data.insert(0, ['rank', 'team', 'sum', 'matches', 'average'])

with open('attendance.csv', 'w') as f:
    write = csv.writer(f)
    for row in data:
        write.writerow(row)
        
attendance = pd.read_csv('attendance.csv')

attendance